In [ ]:
!kaggle datasets download -d jessicali9530/stanford-dogs-dataset
!unzip -q stanford-dogs-dataset.zip -d stanford-dogs-dataset

Dataset URL: https://www.kaggle.com/datasets/jessicali9530/stanford-dogs-dataset
License(s): other
100% 750M/750M [00:05<00:00, 172MB/s]
100% 750M/750M [00:05<00:00, 156MB/s]


In [ ]:
!ls stanford-dogs-dataset

annotations  images


In [ ]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers, models
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
import glob
import xml.etree.ElementTree as ET
from PIL import Image

In [ ]:
# Set random seed for reproducibility
np.random.seed(42)
tf.random.set_seed(42)

In [ ]:
# Define paths (update these paths based on your setup)
dataset_path = 'stanford-dogs-dataset'
images_path = os.path.join(dataset_path, "images")
annotations_path = os.path.join(dataset_path, "annotation")

In [ ]:
import os
import shutil
import random

# Define paths
dataset_dir = 'stanford-dogs-dataset/images/Images'
annotations_dir = 'stanford-dogs-dataset/annotations/Annotation'
filtered_dir = 'filtered-dogs'
filtered_annotations_dir = os.path.join(filtered_dir, 'Annotation')
filtered_images_dir = os.path.join(filtered_dir, 'Images')

# Ensure the filtered directory is clean
if os.path.exists(filtered_dir):
    shutil.rmtree(filtered_dir)
os.makedirs(filtered_annotations_dir)
os.makedirs(filtered_images_dir)

# List all species
all_species = os.listdir(dataset_dir)
print(f'Total species: {len(all_species)}')

# Randomly select 5 species
selected_species = random.sample(all_species, 5)
print(f'Selected species: {selected_species}')

# Copy selected species to the filtered directory
for species in selected_species:
    species_image_path = os.path.join(dataset_dir, species)
    species_annotation_path = os.path.join(annotations_dir, species)
    target_image_path = os.path.join(filtered_images_dir, species)
    target_annotation_path = os.path.join(filtered_annotations_dir, species)

    # Copy image files
    shutil.copytree(species_image_path, target_image_path)

    # Copy annotation files
    shutil.copytree(species_annotation_path, target_annotation_path)

print("Selected species images and annotations have been copied to the filtered directory.")

Total species: 120
Selected species: ['n02112137-chow', 'n02085620-Chihuahua', 'n02106166-Border_collie', 'n02110063-malamute', 'n02111129-Leonberg']
Selected species images and annotations have been copied to the filtered directory.


In [ ]:
# Data generators
datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2,
    rotation_range=40,
    width_shift_range=0.3,
    height_shift_range=0.3,
    shear_range=0.3,
    zoom_range=0.3,
    horizontal_flip=True,
    fill_mode='nearest',
    brightness_range=[0.2, 1.0],
    channel_shift_range=0.2
)

In [ ]:
images_dir = filtered_images_dir  # Use the filtered images directory

In [ ]:
train_generator = datagen.flow_from_directory(
    images_dir,
    target_size=(224, 224),
    batch_size=64,
    class_mode='categorical',
    subset='training'
)

Found 710 images belonging to 5 classes.


In [ ]:
validation_generator = datagen.flow_from_directory(
    images_dir,
    target_size=(224, 224),
    batch_size=64,
    class_mode='categorical',
    subset='validation'
)

Found 176 images belonging to 5 classes.


In [ ]:
test_datagen = ImageDataGenerator(rescale=1./255)
test_generator = test_datagen.flow_from_directory(
    images_dir,
    target_size=(224, 224),
    batch_size=64,
    class_mode='categorical'
)

Found 886 images belonging to 5 classes.


In [ ]:
# Print some information about the generators
print(f'Total training samples: {train_generator.samples}')
print(f'Total validation samples: {validation_generator.samples}')
print(f'Total test samples: {test_generator.samples}')

Total training samples: 710
Total validation samples: 176
Total test samples: 886


In [ ]:
import os
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.regularizers import l2
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam

In [ ]:
# Define a custom CNN architecture
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(128, activation='relu', kernel_regularizer=l2(0.01)),
    Dropout(0.5),
    Dense(train_generator.num_classes, activation='softmax')
])

In [ ]:
# Compile the model with a smaller learning rate
optimizer = Adam(learning_rate=0.0001)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping
# Define early stopping
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

In [ ]:
# Train the model
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // validation_generator.batch_size,
    epochs=10,
    callbacks=[early_stopping]
)

Epoch 1/10
11/11 [==============================] - 95s 8s/step - loss: 3.9051 - accuracy: 0.2337 - val_loss: 3.5902 - val_accuracy: 0.2266
Epoch 2/10
11/11 [==============================] - 94s 8s/step - loss: 3.3654 - accuracy: 0.2043 - val_loss: 3.0986 - val_accuracy: 0.2266
Epoch 3/10
11/11 [==============================] - 92s 8s/step - loss: 2.9357 - accuracy: 0.2616 - val_loss: 2.7383 - val_accuracy: 0.2812
Epoch 4/10
11/11 [==============================] - 101s 9s/step - loss: 2.6127 - accuracy: 0.2523 - val_loss: 2.4673 - val_accuracy: 0.2969
Epoch 5/10
11/11 [==============================] - 91s 8s/step - loss: 2.3836 - accuracy: 0.3050 - val_loss: 2.2845 - val_accuracy: 0.3359
Epoch 6/10
11/11 [==============================] - 94s 8s/step - loss: 2.2128 - accuracy: 0.3034 - val_loss: 2.1430 - val_accuracy: 0.3359
Epoch 7/10
11/11 [==============================] - 90s 8s/step - loss: 2.1021 - accuracy: 0.3189 - val_loss: 2.0120 - val_accuracy: 0.3359
Epoch 8/10
11/11 [=

In [ ]:
# Evaluate the model on the test set
loss, accuracy = model.evaluate(test_generator)
print(f'Test accuracy: {accuracy}')

14/14 [==============================] - 34s 2s/step - loss: 1.7951 - accuracy: 0.3589
Test accuracy: 0.3589164912700653
